In [ ]:
import pandas as pd

# libraries to clean the data
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

# To vectorize and split the data appropriately
from sklearn.feature_extraction.text import TfidfVectorizer

#compute cosign similarities tf idf
from sklearn.metrics.pairwise import linear_kernel

from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
fulldata = pd.read_csv("data/MN/2023-2024_93rd_Legislature/csv/MN_bill_texts_preprocessed.csv")
fulldata.columns = ['bill_id', 'text']

In [ ]:
fulldata.dropna(subset=['bill_id', 'text'], inplace=True)
fulldata = fulldata.astype({'bill_id':'int', 'text':'string'})

In [ ]:
stop_words = stopwords.words('english')

        
def remove_stopwords(texts):
    return [word for word in texts.split() if word not in stop_words]

def clean_text(text):
    text = ' '.join(remove_stopwords(text.lower()))
    # remove punctuation
    text = str(re.sub('[,.!?]', '', text))
    #remove the new line shit
    text = text.replace("\u200b", "")
    #remove all numbers
    text = re.sub(r'\d+', '', text)
    #remove a b c 
    text = text.replace(" a ", "")
    text = text.replace(" b ", "")
    text = text.replace(" c ", "")
    return text

fulldata["clean_text"]= fulldata['text'].apply(lambda x:clean_text(x))

In [ ]:
#tf-idf
vectorizer = TfidfVectorizer()
bill_text = fulldata['clean_text']
bill_text = vectorizer.fit_transform(bill_text)

In [ ]:
#change the indices to match the document index you want to compare to
cosine_similarities = linear_kernel(bill_text[5426:5427], bill_text).flatten()
cosine_similarities

In [ ]:
related_docs_indices = cosine_similarities.argsort()[:-7:-1]
related_docs_indices
fulldata.loc[related_docs_indices,"bill_number"]

In [ ]:
#compute cosign similarities word-to-vec’

tokenized_docs = [doc.lower().split() for doc in fulldata["text"]]
model = Word2Vec(tokenized_docs, min_count=1, vector_size=100)

In [ ]:
def get_doc_vector(doc, model):
    doc_vector = np.zeros(model.vector_size)
    word_count = 0
    for word in doc:
        if word in model.wv:
            doc_vector += model.wv[word]
            word_count += 1
    if word_count != 0:
        return doc_vector / word_count
    else:
        return np.zeros(model.vector_size)

doc_vectors = [get_doc_vector(doc, model) for doc in tokenized_docs]

In [ ]:
omni = doc_vectors[5246]
sims = []
def compute_cosin(x,omni):
    cos_sim = dot(x,omni)/(norm(x)*norm(omni))
    return cos_sim
for x in doc_vectors:
    sim = compute_cosin(x,omni)
    sims.append(sim)

related_docs_indices = np.array(sims).argsort()[:-21:-1]
related_docs_indices

fulldata.loc[related_docs_indices,"bill_number"]